In [1]:
import pandas as pd
import numpy as np
import plotly as plt
import plotly.express as px
import io

### Reading in assessment data for every TN Public school for 5 years of historical data
    > Technically 6 but the test was canceled for 2020

In [2]:
# Missing data related to district, school, and enrollment 
data_2017 = pd.read_csv('../data/school_based/2017.csv')
# Missing enrollment data
data_2018 =pd.read_csv('../data/school_based/2018.csv')
data_2019 = pd.read_csv('../data/school_based/2019.csv')
data_2021 = pd.read_csv('../data/school_based/2021.csv')
# Uses different naming conventions than previous years.
# Metric naming will be standardized to align with 2022 conventions
data_2022 = pd.read_csv('../data/school_based/2022.csv')

In [3]:
# List of annual assessment daframes
dfs = [data_2017, data_2018, data_2019, data_2021, data_2022]
df_names = ['year_2017', 'year_2018', 'year_2019', 'year_2021', 'year_2022']

# Initialize a StringIO object to get info in one go
output = io.StringIO()

# Loop through the dfs and their names
for df, name in zip(dfs, df_names):

    # Write the name of the DataFrame to the output
    output.write(f"{name} info:\n")
    
    # Capture the output of the .info() method
    df.info(buf=output)
    
    # Separater
    output.write("\n---\n")

# output 
info_str = output.getvalue()

# Close the StringIO object
output.close()

print(info_str)

year_2017 info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440633 entries, 0 to 440632
Data columns (total 16 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   year             440633 non-null  int64  
 1   system           440633 non-null  int64  
 2   school           440633 non-null  int64  
 3   subject          440633 non-null  object 
 4   grade            440633 non-null  object 
 5   subgroup         440633 non-null  object 
 6   valid_tests      440633 non-null  float64
 7   n_below          440633 non-null  object 
 8   n_approaching    440633 non-null  object 
 9   n_on_track       440633 non-null  object 
 10  n_mastered       440633 non-null  object 
 11  pct_below        440633 non-null  object 
 12  pct_approaching  440633 non-null  object 
 13  pct_on_track     440633 non-null  object 
 14  pct_mastered     440633 non-null  object 
 15  pct_on_mastered  440633 non-null  object 
dtypes: float64(1), int64(3

## Issues
- Figure out to deal with suppressed data and salvage as much student demographic data as you can      
  - `*` = The number of **valid** test scores is less than 10.
  - `**` = Any individual proficiency level is <u>less than</u> **1%** or <u>greater than</u> **99%** of the district average.


### Combine Datasets with more or less the same naming conventions

In [4]:
middle_years = pd.concat([data_2018, data_2019, data_2021])

# Renaming columns in middle years to match 2022 naming conventions.  They mean the exact same thing.
middle_years = middle_years.rename(columns={
    'n_on_track': 'n_met_expectations',
    'pct_on_track': 'pct_met_expectations',
    'n_mastered': 'n_exceeded_expectations',
    'pct_mastered': 'pct_exceeded_expectations',
    'subgroup': 'student_group',
    'pct_on_mastered': 'pct_met_exceeded'
})
middle_years.head()

,year,system,system_name,school,school_name,test,subject,grade,student_group,valid_tests,...,n_met_expectations,n_exceeded_expectations,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded,enrolled,tested,participation_rate
0,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,All Students,43.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN
1,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Economically Disadvantaged,14.0,...,**,**,**,**,**,**,7.1,NaN,NaN,NaN
2,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Non-Black/Hispanic/Native American,43.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN
3,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Non-Economically Disadvantaged,29.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN
4,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Non-English Learners/Transitional 1-4,43.0,...,**,**,**,**,**,**,**,NaN,NaN,NaN


### Combining `middle_years` dataset with 2022 to unify all data except 2017 

In [5]:
# Combine middle years with data_2022 
assessments_unclean = pd.concat([middle_years, data_2022])

# Dropping Enrolled, tested, participation_rate, and n_columns
assessments_unclean = assessments_unclean.drop(['enrolled', 
                                  'tested', 
                                  'participation_rate',
                                  'n_below',
                                  'n_approaching',
                                  'n_met_expectations',
                                  'n_exceeded_expectations'], axis=1)

assessments_unclean.head(n=2)

,year,system,system_name,school,school_name,test,subject,grade,student_group,valid_tests,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded
0,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,All Students,43.0,**,**,**,**,**
1,2018,10,Anderson County,2,Anderson County High School,EOC,Algebra I,10,Economically Disadvantaged,14.0,**,**,**,**,7.1


In [6]:
# Calculate the number of unique systems and schools in each year
unique_counts = assessments_unclean.groupby('year').agg({'system': pd.Series.nunique, 'school': pd.Series.nunique}).reset_index()

# Melt the DataFrame to get it in the right format for plotting
melted_counts = unique_counts.melt(id_vars='year', var_name='type', value_name='count')

# Create a bar chart with Plotly
fig = px.bar(melted_counts, x='year', y='count', color='type', barmode='group', title='Unique Counts of Systems and Schools per Year')

# Show the chart
fig.show()

### Fix Missing info in 2017 - I am going to drop this for now and focus on the later years.  There is some mislabeled data and a bit of ambiguity about which test belongs with which subject.  I emailed TDOE, but I'm gonna leave it alone until I hear back.

In [7]:
# Create a unique mapping of system & school for the `assessments_unclean` dataframe.
# This is due to there not being a name associated with the numbers
mapping_assessments = assessments_unclean.drop_duplicates(subset=['system', 'school'])

# Merge the 2017 with the mapping
data_2017 = pd.merge(data_2017, mapping_assessments[['system', 'system_name', 'school', 'school_name']], on=['system', 'school'], how='left')\

# Renaming 2017 with standard naming conventions and dropping unneccissary columns
data_2017 = data_2017.rename(
    columns={
    'pct_on_track': 'pct_met_expectations',
    'pct_mastered': 'pct_exceeded_expectations',
    'subgroup': 'student_group',
    'pct_on_mastered': 'pct_met_exceeded'
})

# Dropping the columns not needed for analysis
data_2017 = data_2017.drop([ 
    'n_below',
    'n_approaching',
    'n_on_track',
    'n_mastered'], axis=1)

data_2017.head(n=1)

,year,system,school,subject,grade,student_group,valid_tests,pct_below,pct_approaching,pct_met_expectations,pct_exceeded_expectations,pct_met_exceeded,system_name,school_name
0,2017,10,2,Algebra I,10,All Students,50.0,**,**,**,**,**,Anderson County,Anderson County High School


In [8]:
data_2017['subject'].unique()

array(['Algebra I', 'Algebra II', 'Biology I', 'Chemistry', 'English II',
       'Geometry', 'English III', 'US History', 'English I', 'ELA',
       'Math', 'Science', 'Integrated Math I', 'Integrated Math II',
       'Integrated Math III'], dtype=object)

looks like school `0` occurs as a missing value frequently.  Ill look at that.
- The schools referenced here do not occur in the known districts past 2017.  I'll drop the nan values and hope it doesn't come back to bite me.

In [9]:
# Create new DF of missing 'system name' values
missing_2017 = data_2017[data_2017['school_name'].isna()]

# Drop all but system and school columns
dropped_missing_2017 = missing_2017[['system', 'school']]

# group by system and unique school
dropped_missing_2017 = missing_2017.groupby('system')['school'].unique()
dropped_missing_2017

system
10            [105]
11              [0]
150             [0]
190      [425, 520]
231            [25]
300             [0]
470            [83]
580         [0, 75]
650             [0]
792    [2075, 2760]
794           [170]
800            [35]
820           [200]
830             [0]
860             [0]
985    [8035, 8080]
Name: school, dtype: object

### Subject/Test Cleaning

Looks like I need to break this table down a bit further to aggregate by test and then subject. The data are noisy and I will filter by the aggregated scores reported in each school. The general hueristic is that the majority of students take these tests in a specific grade.  However, some students take the tests earlier or later than their peers.  In these cases, there can be much grade-level suppression.  Therefore, aggregatons are much less suppressed than grade-level analysis.

In [10]:
# Dropping MSAA & Alt
assessments_unclean = assessments_unclean[assessments_unclean["test"].str.contains("MSAA|MSAA/Alt-Science/Social Studies") == False]

# Filtering for "All Grades" aggregate of each subject
assessments_unclean= assessments_unclean[assessments_unclean["grade"].str.contains("All Grades")]

# Removing student_groups that do not have enough valid tests.
assessments_unclean = assessments_unclean[assessments_unclean['pct_below'] != '*']

# Dropping proficeincy categories to explore solely "met" or "did not meet" expectations
# Dropped 'grade' as it's no longer descriptive
assessments_unclean = assessments_unclean.drop(['grade', 'pct_below', 'pct_approaching', 'pct_met_expectations', 'pct_exceeded_expectations'], axis=1)

### Import `elsi_clean.csv` as `elsi` and merge with `assessments`.

In [20]:
elsi = pd.read_pickle('../data/school_based/elsi_clean.pkl')  

### Testing Merge of DFs
> I'm really praying that the `system_name` and `school_name` are standardized.  Otherwise, I'll have to reconstruct `system` and `school` from `assessments` to their standardised titles as used in the `elsi` data.

assessments_unclean

In [48]:
# List of comumns
assessments_unclean.columns.tolist()

['year',
 'system_name',
 'school_name',
 'system',
 'school',
 'test',
 'subject',
 'student_group',
 'valid_tests',
 'pct_met_exceeded']

In [44]:
# List of new column order
unclean_order = (['year',
                  'system_name',
                  'school_name',
                  'system',
                  'school',
                  'test',
                  'subject',
                  'student_group',
                  'valid_tests',
                  'pct_met_exceeded'
                ])
# Reorder columns and sort
assessments_unclean = assessments_unclean[unclean_order].sort_values(['year', 'system_name', 'school_name'])

In [45]:
assessments_unclean.head()

,year,system_name,school_name,system,school,test,subject,student_group,valid_tests,pct_met_exceeded
590415,2018,Achievement School District,Aspire Coleman,985,8050,TNReady,ELA,All Students,321.0,11.2
590418,2018,Achievement School District,Aspire Coleman,985,8050,TNReady,ELA,Black or African American,299.0,11.4
590420,2018,Achievement School District,Aspire Coleman,985,8050,TNReady,ELA,Black/Hispanic/Native American,312.0,11.5
590422,2018,Achievement School District,Aspire Coleman,985,8050,TNReady,ELA,Economically Disadvantaged,222.0,9.5
590425,2018,Achievement School District,Aspire Coleman,985,8050,TNReady,ELA,English Learners with Transitional 1-4,11.0,18.2


Hopfully I can merge on the names and not the codes for system and school.  If it doesn't work, I'll have to try to standardize the naming.


In [46]:
# List of elsi columns
elsi.columns.tolist()

['year',
 'system_name',
 'school_name',
 'system',
 'school',
 'school_lvl',
 'tot_enrolled',
 'fte_teachers',
 'stu_tchr_ratio',
 'school_type',
 'magnet',
 'charter',
 'virtual',
 'title_1',
 'lat',
 'long',
 'locale']

specify order of elsi columns

In [47]:
# List of desired elsi column order
elsi_order = ([
 'year',
 'system_name',
 'school_name',
 'system',
 'school',
 'school_lvl',
 'tot_enrolled',
 'fte_teachers',
 'stu_tchr_ratio',
 'school_type',
 'magnet',
 'charter',
 'virtual',
 'title_1',
 'lat',
 'long',
 'locale'
])

# Reorder columns and sort
elsi = elsi[elsi_order].sort_values(['year', 'system_name', 'school_name'])

In [43]:
elsi.head()

,year,system_name,school_name,system,school,school_lvl,tot_enrolled,fte_teachers,stu_tchr_ratio,school_type,magnet,charter,virtual,title_1,lat,long,locale
917,2018,Achievement School District,Aspire Coleman,TN-00985,TN-00985-8050,Elementary,570,27.5,20.73,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.2141,-89.923641,11-City: Large
920,2018,Achievement School District,Aspire Hanley Elementary,TN-00985,TN-00985-8025,Elementary,513,19.5,26.31,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.113582,-89.976593,11-City: Large
919,2018,Achievement School District,Aspire Middle School,TN-00985,TN-00985-8024,Middle,225,13.5,16.67,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.113692,-89.976286,11-City: Large
157,2018,Achievement School District,Brick Church: A LEAD Public School,TN-00985,TN-00985-8005,Middle,327,13.8,23.7,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,36.220505,-86.781436,11-City: Large
368,2018,Achievement School District,Cornerstone Prep - Lester Campus,TN-00985,TN-00985-8010,Elementary,416,28.3,14.7,1-Regular school,2-No,1-Yes,NOTVIRTUAL,1-Yes,35.141146,-89.963536,11-City: Large


### Exploring Fully Suppressed outliers (Do this after the great merging)
>  ``**`` suppresion is applied to the school level subject scores if that school performes above 95% or below 5% of a district's average.  


> Subject matter expert confirmed that it is the majority case that schools scores are suppressed for falling under 5% of the district average.  There are only a 'handful' of scores suppressed for averages greater than 95% of the district.

In [12]:
# full_suppression = assessments_unclean.loc[(assessments_unclean['pct_met_exceeded'] == '**')]
# full_suppression.drop(['system','school','pct_met_exceeded'], axis=1)

In [13]:
# # Calculating total percentage of not met for student_groups per assessment
# #bob = assessments_unclean["pct_not_met"] = assessments_unclean['pct_met_exceeded'] - 1

# assessments_unclean

In [14]:
# assessments_unclean = assessments_unclean.set_index('year')

In [15]:
# # Look at the subjects for each test.    
# test_subjects = assessments_unclean[['test', 'subject', 'student_group']]
# test_subjects.head()

In [16]:
# pivot_table = assessments_unclean.pivot_table(index='test', columns='subject', aggfunc='size', fill_value=0)
# pivot_table